# <font color=#B40431>Ensemble Methods<font>

## <font color=#424242>A Sexy Machine Learning Strategy<font>

### Dataset

We want to assess the credit risk of someone using their finantial history. Basically, we'll use 20 finantial attributes to classify people as good or bad credit risks. 

The dataset is available online:
https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29

In [16]:
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from IPython.display import Image  
import pydotplus
from sklearn.externals.six import StringIO 

In [17]:
project_dir = os.path.dirname(os.getcwd())
path_data = os.path.join(project_dir, 'data', 'cars_dataset.csv')

In [18]:
df_data = pd.read_csv(path_data, sep=';', dtype={'cars':str, 'doors':str, 'seats':str})
df_data.set_index('cars', inplace=True)
df_data.head()

,buying,maint,doors,seats,lugg_boot,safety,target
cars,,,,,,,
1,high,low,3,2,med,med,unacc
2,med,low,5,5,small,high,acc
3,med,med,5,9,big,high,acc
4,high,high,3,18,big,med,acc
5,high,high,3,5,small,low,unacc


In [19]:
inputs = df_data[['buying', 'maint', 'doors', 'seats', 'lugg_boot', 'safety']]
targets = df_data[['target']]
inputs = pd.get_dummies(inputs)

In [20]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(inputs, targets)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [21]:
y_pred=clf.predict(inputs)

In [22]:
y_pred

array(['unacc', 'acc', 'acc', 'acc', 'unacc', 'unacc'], dtype=object)

In [23]:
targets

,target
cars,
1,unacc
2,acc
3,acc
4,acc
5,unacc
6,unacc


In [26]:
export_graphviz(clf,out_file=os.path.join(project_dir, 'tree.dot'))

In [28]:
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

InvocationException: GraphViz's executables not found